In [108]:
import pandas as pd
import numpy as np
import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AdamW
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import pickle
import ast
import statistics as s

In [71]:
# load word2vec model
#w2v_model = ...
# load BERT model
#BERT_model = ...
# load dataframe
#df = ...

tokenizer = BertTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
BERT_model = BertForMaskedLM.from_pretrained("GroNLP/bert-base-dutch-cased")
#w2v_model.intersect_word2vec_format(r"combined-160.txt", binary=False, lockf=1.0)
# https://github.com/clips/dutchembeddings

df = pd.read_csv('preprocessed_df100.csv')

In [72]:
def finetune_word2vec(train, window=5):
    sentences = train.split('.')
    tokenizer = RegexpTokenizer(r'\w+')
    sentences = [tokenizer.tokenize(i) for i in sentences]
    total_examples = len(sentences)
    
    model_w2v = Word2Vec(size=160, min_count=1, window=window)
    model_w2v.build_vocab(sentences)
    total_examples = model_w2v.corpus_count
    model = KeyedVectors.load_word2vec_format(r"combined-160.txt", binary=False)
    model_w2v.build_vocab([list(model.vocab.keys())], update=True)
    model_w2v.intersect_word2vec_format(r"combined-160.txt", binary=False, lockf=1.0)
    model_w2v.train(sentences, total_examples=total_examples, epochs=model_w2v.iter)
    return model_w2v

train = df['gt text'][0]
word2vec_model = finetune_word2vec(train)

/home/nvanthof/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  del sys.path[0]


In [7]:
df.columns
print(df.index.values.tolist())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [4]:
# set hyperparameters for validation
#topn_detection = 1000
#topn_correction = 1000
#window = 5

In [59]:
def list_merger(lists):
    new_list = []
    for elem in lists:
        new_list = new_list + elem
    return new_list

In [41]:
#lijst = [['aa','bb','cc'],['dd','ee','ff'],[7,8,9]]
lijst = [['engelant', 'ampc'], ['dublin', 'den', '24', 'maert'], ['tegens', 'de', 'charters', 'van', 'dese', 'stadt'], []]
lijst = list_merger(lijst)
print(lijst)

1: ['engelant', 'ampc']
ELEM: ['engelant', 'ampc']
ELEM: ['dublin', 'den', '24', 'maert']
ELEM: ['tegens', 'de', 'charters', 'van', 'dese', 'stadt']
ELEM: []
['engelant', 'ampc', 'dublin', 'den', '24', 'maert', 'tegens', 'de', 'charters', 'van', 'dese', 'stadt']


In [54]:
string = "[['engelant', 'ampc'], ['dublin', 'den', '24', 'maert'], ['tegens', 'de', 'charters', 'van', 'dese', 'stadt'], []]"
lijst = ast.literal_eval(string)
print(type(lijst))
print(lijst)
print(lijst[0][0])

<class 'list'>
[['engelant', 'ampc'], ['dublin', 'den', '24', 'maert'], ['tegens', 'de', 'charters', 'van', 'dese', 'stadt'], []]
engelant


In [117]:
# skiplist (words that should not be corrected: names)
with open("ocr_names.txt", "rb") as fp:   # Unpickling
    ocr_names = pickle.load(fp)

ocr_names = []
for name in ocr_names:
    if len(name) >= 5:
        ocr_names.append(name)


In [118]:
#detection and correction validation

detection_df = pd.DataFrame()
error_positions = [] # the position of a word when it is an error in predictions
non_error_positions = [] # the position of a word when it is not an error in predictions
right_token_positions = []

def detection_and_correction_word2vec(row, w2v_model, ocr_names, window=5, topn_detection=1000):
    if row['set'] != 'val':
        return np.nan
    else:
        print('Val')
        OCR_text = row['aligned_OCR_sentences']
        GT_text = row['aligned_GT_sentences']
        print(OCR_text)
        OCR_text = ast.literal_eval(OCR_text)
        GT_text = ast.literal_eval(GT_text)
        OCR_text = list_merger(OCR_text)
        GT_text = list_merger(GT_text)
        OCR_text = OCR_text[:10]
        print(OCR_text)
        window_range = list(range(0,window))
        window_range = np.array(window_range) - ((window - 1) / 2)
        error_positions_doc = []
        non_error_positions_doc = []
        right_token_positions_doc = []
        for i in range(len(OCR_text)):
            if (OCR_text[i] in ocr_names) or (OCR_text[i].isalpha() == False) or (len(OCR_text[i]) <= 2):
                continue
            error = True
            if OCR_text[i] == GT_text[i]:
                error = False
            context = []
            for j in window_range:
                if (i+j >= 0) and (i+j < len(OCR_text)) and j != 0:
                    context.append(OCR_text[i+int(j)])
                else:
                    pass
            predictions = []
            # calculate positions detection task
            for prediction in w2v_model.predict_output_word(context, topn=topn_detection):
                predictions.append(prediction[0])
            try:
                position = predictions.index(OCR_text[i])
            except ValueError:
                position = topn_detection
            if error == True:
                error_positions_doc.append(position)
            elif error == False:
                non_error_positions_doc.append(position)
            # calculate positions correction task
            if error == True:
                try:
                    right_token_position = predictions.index(GT_text[i])
                    right_token_positions_doc.append(right_token_position)
                except ValueError:
                    right_token_positions_doc.append(topn_detection)
                right_token_positions.append(right_token_positions_doc)    
            
            
    #return error_positions, non_error_positions
    error_positions.append(error_positions_doc)
    non_error_positions.append(non_error_positions_doc)
    
    
    
#for index, row in df.iterrows():
#    detection_word2vec(row)
# df.loc[70]
fake_test_list_GT_aligned = """Een koekenpan of kortweg pan is een platte pan met een lang handvat.
De pan ontleent zijn naam aan het feit dat in zo'n pan pannenkoeken worden gebakken. Ook ander voedsel, zoals vlees, wordt in een koekenpan gebraden"""
fake_test_list_OCR_aligned = """Een hoekenpan of hortweg pan is een hlatte pan met een hang handvat.
De pan ontleent zijn naam haan het heit dat in ho'n pan pannenkoeken horden gebakken. Ook hnder voedsel, zoals vlees, word in een koekenpan gebraden"""
fake_test_list_GT_aligned = fake_test_list_GT_aligned.split('.')
fake_test_list_OCR_aligned = fake_test_list_OCR_aligned.split('.')
fake_test_list_GT_aligned = [x.split(' ') for x in fake_test_list_GT_aligned]
fake_test_list_OCR_aligned = [x.split(' ') for x in fake_test_list_OCR_aligned]
d = {'aligned_OCR_sentences': [str(fake_test_list_OCR_aligned)], 'aligned_GT_sentences': [str(fake_test_list_GT_aligned)], 'set': ['val']}
df_probeer = pd.DataFrame(data=d)
detection_and_correction_word2vec(df_probeer.loc[0], word2vec_model, ocr_names)
print(error_positions)
print(non_error_positions)
error_positions = list_merger(error_positions)
non_error_positions = list_merger(non_error_positions)
right_token_positions = list_merger(right_token_positions)
print('errors:', s.mean(error_positions))
print('non errors:', s.mean(non_error_positions))
print('right token in prediction:', s.mean(right_token_positions))



Val
[['Een', 'hoekenpan', 'of', 'hortweg', 'pan', 'is', 'een', 'hlatte', 'pan', 'met', 'een', 'hang', 'handvat'], ['\nDe', 'pan', 'ontleent', 'zijn', 'naam', 'haan', 'het', 'heit', 'dat', 'in', "ho'n", 'pan', 'pannenkoeken', 'horden', 'gebakken'], ['', 'Ook', 'hnder', 'voedsel,', 'zoals', 'vlees,', 'word', 'in', 'een', 'koekenpan', 'gebraden']]
['Een', 'hoekenpan', 'of', 'hortweg', 'pan', 'is', 'een', 'hlatte', 'pan', 'met']
[[1000, 1000, 1000]]
[[1000, 1000, 1000, 1000, 5]]
errors: 1000
non errors: 801
right token in prediction: 1000


In [37]:
text = "Dit1 is2 een3 zin4 die5 lang6 genoeg7 is8"
text = text.split(' ')

window = 5
window_range = list(range(0,window))
print(window_range)
window_range = np.array(window_range) - ((window - 1) / 2)
print(window_range)

context = []

for i in range(len(text)):
    print(i)
    print(text[i])
    context = []
    for j in window_range:
        if (i+j >= 0) and (i+j < len(text)) and j != 0:
            context.append(text[i+int(j)])
        else:
            pass
    print('context:', context)
    



[0, 1, 2, 3, 4]
[-2. -1.  0.  1.  2.]
0
Dit1
context: ['is2', 'een3']
1
is2
context: ['Dit1', 'een3', 'zin4']
2
een3
context: ['Dit1', 'is2', 'zin4', 'die5']
3
zin4
context: ['is2', 'een3', 'die5', 'lang6']
4
die5
context: ['een3', 'zin4', 'lang6', 'genoeg7']
5
lang6
context: ['zin4', 'die5', 'genoeg7', 'is8']
6
genoeg7
context: ['die5', 'lang6', 'is8']
7
is8
context: ['lang6', 'genoeg7']


In [103]:
word2vec_model.predict_output_word(['Een', 'of', 'kortweg'], topn=10)

[('den', 7.57105e-07),
 ('ende', 7.248402e-07),
 ('de', 7.188712e-07),
 ('noch', 7.1387456e-07),
 ('is', 7.0737497e-07),
 ('hebben', 7.070108e-07),
 ('vyand', 7.065082e-07),
 ('eenige', 7.0626095e-07),
 ('met', 7.061281e-07),
 ('wt', 7.042607e-07)]

In [102]:
word2vec_model.wv.most_similar('papier')

[('karton', 0.8278681039810181),
 ('inkt', 0.8026106357574463),
 ('rubberdoek', 0.7930652499198914),
 ('krantenpapier', 0.792934775352478),
 ('perkament', 0.7911171913146973),
 ('vloeipapier', 0.7774406671524048),
 ('vel', 0.7763333320617676),
 ('handgeschept', 0.771336555480957),
 ('potlood', 0.760221540927887),
 ('bedrukken', 0.7578753232955933)]